In [1]:
import pandas as pd

In [25]:
df = pd.read_excel('../chat_bot_aep/data/total.xlsx', dtype={'point': str})

In [144]:
filter_df_2 = df[~df['document'].isin(['НП-001-15', 'НП-031-01', 'НП-044-18', 'НП-045-18', 'НП-068-05', 'НП-089-15', 'НП-104-18'])][['document', 'description']].reset_index(drop=True)

In [146]:
filter_df_2['merge_document_description'] = filter_df_2['document'] + '. ' + filter_df_2['description'] 

In [148]:
filter_df_2

,document,description,merge_document_description
0,НП-005-16,Настоящие федеральные нормы и правила в област...,НП-005-16. Настоящие федеральные нормы и прави...
1,НП-005-16,Настоящее Положение устанавливает:\nкритерии о...,НП-005-16. Настоящее Положение устанавливает:\...
2,НП-005-16,Требования настоящего Положения распространяют...,НП-005-16. Требования настоящего Положения рас...
3,НП-005-16,Требования настоящего Положения должны выполня...,НП-005-16. Требования настоящего Положения дол...
4,НП-005-16,"Критериями объявления состояния ""Аварийная гот...","НП-005-16. Критериями объявления состояния ""Ав..."
...,...,...,...
5026,НП-105-18,Отчетная документация должна оформляться для к...,НП-105-18. Отчетная документация должна оформл...
5027,НП-105-18,Результаты контроля каждым методом оформляются...,НП-105-18. Результаты контроля каждым методом ...
5028,НП-105-18,В отчетной документации по контролю металла св...,НП-105-18. В отчетной документации по контролю...
5029,НП-105-18,Отчетная документация должна оформляться на бу...,НП-105-18. Отчетная документация должна оформл...


In [164]:
import requests
import uuid

def get_token(auth_token, scope='GIGACHAT_API_PERS'):
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «GIGACHAT_API_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    # Создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())

    # API URL
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    # Заголовки
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    # Тело запроса
    payload = {
        'scope': scope
    }

    try:
        # Делаем POST запрос с отключенной SSL верификацией
        # (можно скачать сертификаты Минцифры, тогда отключать проверку не надо)
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return -1

In [166]:
auth = 'M2I0MDBkMGMtNjViZC00MGE4LTgxNWMtNGJjYjJkNjNlYTU3OmQ5YzNkODY3LTZiNGMtNGRmYi1hYTE5LWJiMGI2OTE2MzFkMg=='
response = get_token(auth)
if response != 1:
  print(response.text)
  giga_token = response.json()['access_token']

{"access_token":"eyJjdHkiOiJqd3QiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwiYWxnIjoiUlNBLU9BRVAtMjU2In0.QMkRSznAj5vnOGsmRG1KCGc7d9ROSGpd_C5K08yCqxFHuCRKfJWE685uqD6FxwL3IkFOVTVLcjI16oO2zTgEUZvE3Y-KzU0lTJCEXnFKJQs1kZ862--3QwI0eQ5eM8z1BJ0Rwuyj-ACP1VPWp32xJn0GFZ11njcAWHGNK1HYsS2GIxTyoUSwfCHtRz03gqXQ5CG7WXbis-4F_06uCOBVUXeWxox8IIqNVcwxjPmdM6QxrfIEbCwYqxw7J20RS-UBax3NSurodrUH6B9PVBNvpckrddkVh_birRZSqz0kRW_dg2CTWPCf_TJC9m1DI1B7qtRq9_arFb2CDI4_drGgrw.6mbzZT8VIO7pUeMb6dT2ig.0wpGNgLtzAEOln-2Rk_D3YIghe4_hb3R58V_ksZo9EUn20RPgpdDlaaE8hvP3tyRZUqA1_rlYjQfkreH5ZehyTHi3CLcB1GgygzAqJCLRaHyMLrAbE9rw7jFsPpiL3NjVK5x19aY6z4OjT7017FyOQuYwAQUtPlTn0cWGxB6GqjAV2oeYF-ugcOs9bXmE6aR3Y0OPfWzXDSO4HeTojCD80npK7t5xKW2fd7zPMx4bkiXuBhfM_yclGDw8Kygu7P38MpPUKjWW_RmuuSiJzCcaRcI49X_3wauUlG8vW6aJz6TJMg7gbKokifyJKxRzj7IwLU9xoWkwNZnMuIPRa9R8mDLEnYR_ChcNoJttFwnBEUhLVfoGGj6Ed4raUtyrLwNTj-YDeUt-IhsfGYF472ZdBSYOA511R31tHGL5eDvHOC9mjs7gBHscrbWJPpPQgHPUT88Eo75fu0QgXGw1GLRyivhQa4dNzc-tedXjzgaKDddumDyof-0YxMdqM-X6R5pvSIoh6nt_lkBqtBQ4ehnOfFtPIimI

In [168]:
url = "https://gigachat.devices.sberbank.ru/api/v1/models"

payload={}
headers = {
  'Accept': 'application/json',
  'Authorization': f'Bearer {giga_token}'
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

print(response.text)

{"object":"list","data":[{"id":"GigaChat","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-2","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-2-Max","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-2-Pro","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Max","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Plus","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Pro","object":"model","owned_by":"salutedevices","type":"chat"}]}


In [170]:
import json

def get_chat_completion(auth_token, user_message):
    """
    Отправляет POST-запрос к API чата для получения ответа от модели GigaChat.

    Параметры:
    - auth_token (str): Токен для авторизации в API.
    - user_message (str): Сообщение от пользователя, для которого нужно получить ответ.

    Возвращает:
    - str: Ответ от API в виде текстовой строки.
    """
    # URL API, к которому мы обращаемся
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    # Подготовка данных запроса в формате JSON
    payload = json.dumps({
        "model": "GigaChat",  # Используемая модель
        "messages": [
            {
                "role": "user",  # Роль отправителя (пользователь)
                "content": user_message  # Содержание сообщения
            }
        ],
        "temperature": 1,  # Температура генерации
        "top_p": 0.1,  # Параметр top_p для контроля разнообразия ответов
        "n": 1,  # Количество возвращаемых ответов
        "stream": False,  # Потоковая ли передача ответов
        "max_tokens": 512,  # Максимальное количество токенов в ответе
        "repetition_penalty": 1,  # Штраф за повторения
        "update_interval": 0  # Интервал обновления (для потоковой передачи)
    })

    # Заголовки запроса
    headers = {
        'Content-Type': 'application/json',  # Тип содержимого - JSON
        'Accept': 'application/json',  # Принимаем ответ в формате JSON
        'Authorization': f'Bearer {auth_token}'  # Токен авторизации
    }

    # Выполнение POST-запроса и возвращение ответа
    try:
        response = requests.request("POST", url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        # Обработка исключения в случае ошибки запроса
        print(f"Произошла ошибка: {str(e)}")
        return -1

In [172]:
from tqdm import tqdm
import time
import warnings

# Отключаем все предупреждения
warnings.filterwarnings('ignore')


In [120]:
# filter_df['question'] = None

In [176]:
for i in tqdm(range(2851, len(filter_df_2))):

    answer = 'У меня есть пункт правил, придумай вопрос, на который будет такой ответ: ' + f'"{filter_df_2.iloc[i]['merge_document_description']}"'
    question = get_chat_completion(giga_token, answer)

    filter_df_2.loc[i, 'question'] = question.json()['choices'][0]['message']['content']

100%|██████████| 2180/2180 [23:23<00:00,  1.55it/s]


In [177]:
filter_df_2

,document,description,merge_document_description,question
0,НП-005-16,Настоящие федеральные нормы и правила в област...,НП-005-16. Настоящие федеральные нормы и прави...,Какой документ регламентирует порядок разработ...
1,НП-005-16,Настоящее Положение устанавливает:\nкритерии о...,НП-005-16. Настоящее Положение устанавливает:\...,Какой документ регламентирует порядок объявлен...
2,НП-005-16,Требования настоящего Положения распространяют...,НП-005-16. Требования настоящего Положения рас...,К какому документу предъявляются требования НП...
3,НП-005-16,Требования настоящего Положения должны выполня...,НП-005-16. Требования настоящего Положения дол...,"Какой документ содержит требования, которые не..."
4,НП-005-16,"Критериями объявления состояния ""Аварийная гот...","НП-005-16. Критериями объявления состояния ""Ав...","Что является критериями объявления состояния ""..."
...,...,...,...,...
5026,НП-105-18,Отчетная документация должна оформляться для к...,НП-105-18. Отчетная документация должна оформл...,Какой пункт правил требует оформления отчетной...
5027,НП-105-18,Результаты контроля каждым методом оформляются...,НП-105-18. Результаты контроля каждым методом ...,Какой документ оформляется по результатам конт...
5028,НП-105-18,В отчетной документации по контролю металла св...,НП-105-18. В отчетной документации по контролю...,"Какой пункт правил требует, чтобы в отчетной д..."
5029,НП-105-18,Отчетная документация должна оформляться на бу...,НП-105-18. Отчетная документация должна оформл...,"""Какой формат документации требуется для отчет..."


In [184]:
df_with_question = pd.concat([filter_df, filter_df_2])
df_with_question

,document,description,merge_document_description,question
0,НП-001-15,Большой аварийный выброс - выброс радиоактивны...,НП-001-15. Большой аварийный выброс - выброс р...,Что является критерием для выполнения мер защи...
1,НП-001-15,Общие положения устанавливают цели и основные ...,НП-001-15. Общие положения устанавливают цели ...,Какой документ устанавливает основные принципы...
2,НП-001-15,Настоящие Общие положения распространяются на ...,НП-001-15. Настоящие Общие положения распростр...,Каким образом НП-001-15 устанавливает требован...
3,НП-001-15,Порядок приведения атомных станций в соответст...,НП-001-15. Порядок приведения атомных станций ...,Кем определяется порядок приведения атомных ст...
4,НП-001-15,"Перечень сокращений приведен в приложении № 1,...",НП-001-15. Перечень сокращений приведен в прил...,Какой документ содержит перечень сокращений и ...
...,...,...,...,...
5026,НП-105-18,Отчетная документация должна оформляться для к...,НП-105-18. Отчетная документация должна оформл...,Какой пункт правил требует оформления отчетной...
5027,НП-105-18,Результаты контроля каждым методом оформляются...,НП-105-18. Результаты контроля каждым методом ...,Какой документ оформляется по результатам конт...
5028,НП-105-18,В отчетной документации по контролю металла св...,НП-105-18. В отчетной документации по контролю...,"Какой пункт правил требует, чтобы в отчетной д..."
5029,НП-105-18,Отчетная документация должна оформляться на бу...,НП-105-18. Отчетная документация должна оформл...,"""Какой формат документации требуется для отчет..."


In [188]:
df_with_question.to_excel('../chat_bot_aep/data/df_with_question.xlsx', sheet_name='Sheet1')

In [93]:
# answer = get_chat_completion(giga_token, ques)

In [110]:
# print(answer.json()['choices'][0]['message']['content'])